In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

In [2]:
# ======================
# CONFIG
# ======================
DATA_DIR = "cnn_dataset"
IMG_SIZE = 224
BATCH_SIZE = 32
# EPOCHS = 25
EPOCHS = 30
# LR = 3e-4
LR = 1e-4
# LR = 5e-5
# LR = 3e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

SAVE_PATH = "resnet18_best_finetuned.pth"

cuda


In [3]:
# ======================
# TRANSFORMS
# ======================
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [4]:
# ======================
# DATASETS
# ======================
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", train_tfms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val", val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print("Classes:", train_ds.classes)

NUM_CLASSES = len(train_ds.classes)


Classes: ['1509', 'IRRI-6', 'Super White']


In [5]:
# ======================
# MODEL (ResNet18)
# ======================
model = models.resnet18(weights="DEFAULT")

# unfreeze all layers
for param in model.parameters():
    param.requires_grad = True

# Replace classifier
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, NUM_CLASSES)

model = model.to(DEVICE)

# ======================
# LOSS / OPTIMIZER
# ======================
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

# optimizer = optim.Adam(model.fc.parameters(), lr=LR)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [6]:
# ======================
# TRAIN LOOP
# ======================
best_acc = 0

for epoch in range(EPOCHS):

    # -------- TRAIN --------
    model.train()
    correct, total = 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):

        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, preds = outputs.max(1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # -------- VAL --------
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:

            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)

            _, preds = outputs.max(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total

    print(f"\nTrain Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), SAVE_PATH)
        print("✅ Best model saved")

print("\n🔥 Training finished")
print("Best Val Accuracy:", best_acc)


Epoch 1/30: 100%|██████████| 282/282 [01:58<00:00,  2.38it/s]



Train Acc: 0.8994 | Val Acc: 0.9500
✅ Best model saved


Epoch 2/30: 100%|██████████| 282/282 [01:57<00:00,  2.40it/s]



Train Acc: 0.9472 | Val Acc: 0.9500


Epoch 3/30: 100%|██████████| 282/282 [02:06<00:00,  2.23it/s]


...


Train Acc: 0.9932 | Val Acc: 0.9644


Epoch 29/30: 100%|██████████| 282/282 [00:46<00:00,  6.01it/s]



Train Acc: 0.9931 | Val Acc: 0.9833
✅ Best model saved


Epoch 30/30: 100%|██████████| 282/282 [00:46<00:00,  6.02it/s]



Train Acc: 0.9917 | Val Acc: 0.9789

🔥 Training finished
Best Val Accuracy: 0.9833333333333333
